In [1]:
from pipeline import *

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from catboost import CatBoostClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import set_config
from sklearn import metrics

np.random.seed(101)
pd.set_option('display.max_columns', None)
set_config(transform_output="pandas")

In [3]:
df = pd.read_parquet(r'C:\Users\minht\Desktop\MLOps\data\phase-2\prob-2\raw_train.parquet')

In [4]:
df = df.sample(frac=1)

In [5]:
df.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,label
34187,0.081963,tcp,-,FIN,22.0,14.0,1470.0,1728.0,31.0,29.0,1.370374e+05,156656.046900,5.0,4.0,3.903000,6.248692,399.950226,14.982982,255.0,1.213830e+09,3.363719e+09,255.0,0.000707,0.000582,0.000125,67.0,123.0,0.0,0.0,9.0,0.0,4.0,1.0,1.0,3.0,0.0,0.0,0.0,3.0,16.0,0.0,Normal
28653,0.000010,udp,dns,INT,2.0,0.0,114.0,0.0,254.0,0.0,4.560000e+07,0.000000,0.0,0.0,0.010000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,4.0,2.0,5.0,4.0,4.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,Other
7570,0.796955,tcp,-,FIN,10.0,10.0,608.0,646.0,254.0,252.0,5.500938e+03,5842.237305,2.0,3.0,88.550556,80.579555,4858.346581,127.869945,255.0,2.375786e+09,2.121203e+09,255.0,0.159596,0.070978,0.088618,61.0,65.0,0.0,0.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,Exploits
50165,0.000009,udp,dns,INT,2.0,0.0,114.0,0.0,254.0,0.0,5.066666e+07,0.000000,0.0,0.0,0.009000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,33.0,2.0,26.0,26.0,13.0,33.0,0.0,0.0,0.0,26.0,33.0,0.0,Other
55555,0.376387,tcp,-,FIN,10.0,6.0,1612.0,268.0,254.0,252.0,3.084060e+04,4761.057129,2.0,1.0,38.067556,59.815801,2007.245167,88.242922,255.0,1.663055e+09,3.271902e+08,255.0,0.136073,0.077299,0.058774,161.0,45.0,0.0,0.0,7.0,1.0,2.0,2.0,1.0,7.0,0.0,0.0,0.0,2.0,7.0,0.0,Denial of Service


In [6]:
fins = ['feature7', 'feature9', 'feature26', 'feature3', 'feature2',
       'feature35', 'feature40', 'feature8', 'feature27', 'feature30',
       'feature34', 'feature23', 'feature12', 'feature11', 'feature39',
       'feature1', 'feature33', 'feature10', 'feature4', 'feature31',
       'feature15']

nums = [
    "feature1",    "feature5",    "feature6",    "feature7",    "feature8",    "feature9",    "feature10",    "feature11",
    "feature12",    "feature13",    "feature14",    "feature15",    "feature16",    "feature17",    "feature18",    "feature19",
    "feature20",    "feature21",    "feature22",    "feature23",    "feature24",    "feature25",    "feature26",    "feature27",
    "feature28",    "feature29",    "feature30",    "feature31",    "feature32",    "feature33",    "feature34",    "feature35",
    "feature36",    "feature37",    "feature38",    "feature39",    "feature40",    "feature41"
]

cats = [
    "feature2", "feature3", "feature4"
]

nums = [i for i in nums if i in fins]
cats = [i for i in cats if i in fins]

In [7]:
import re

nums = [int(re.findall(r'\d+', num)[0]) - 1 for num in nums]
cats = [int(re.findall(r'\d+', cat)[0]) - 1 for cat in cats]
fins = nums + cats

In [8]:
X_train, X_val, y_train, y_val = train_test_split(df.drop(columns=['label']).values, df['label'].values, test_size=0.2, random_state=101)

In [9]:
s = Pipeline([
    ('feature_select', FeatureSelector(fins))
    ]
)

In [10]:
s.fit(X_train, y_train)

Pipeline(steps=[('feature_select',
                 FeatureSelector(_feature_names=[0, 6, 7, 8, 9, 10, 11, 14, 22,
                                                 25, 26, 29, 30, 32, 33, 34, 38,
                                                 39, 1, 2, 3]))])

In [11]:
import pickle

with open(r'C:\Users\minht\Desktop\MLOps\src\phase2\models\pipeline_2.pkl', 'wb') as f:
    pickle.dump(s, f)

In [12]:
X_train = s.transform(X_train)
X_val = s.transform(X_val)

In [13]:
model = CatBoostClassifier(
    iterations=2000,
    random_seed=101,
    learning_rate=0.02,
    eval_metric='Accuracy',
    use_best_model=True
)

In [14]:
model.fit(X_train, y_train, eval_set=(X_val, y_val), cat_features=[18, 19, 20])

0:	learn: 0.7583482	test: 0.7554370	best: 0.7554370 (0)	total: 156ms	remaining: 5m 12s
1:	learn: 0.7570343	test: 0.7556795	best: 0.7556795 (1)	total: 258ms	remaining: 4m 17s
2:	learn: 0.7577822	test: 0.7562455	best: 0.7562455 (2)	total: 362ms	remaining: 4m 1s
3:	learn: 0.7584290	test: 0.7556795	best: 0.7562455 (2)	total: 463ms	remaining: 3m 50s
4:	learn: 0.7580854	test: 0.7567305	best: 0.7567305 (4)	total: 562ms	remaining: 3m 44s
5:	learn: 0.7592780	test: 0.7561646	best: 0.7567305 (4)	total: 664ms	remaining: 3m 40s
6:	learn: 0.7598642	test: 0.7568922	best: 0.7568922 (6)	total: 767ms	remaining: 3m 38s
7:	learn: 0.7587524	test: 0.7564071	best: 0.7568922 (6)	total: 869ms	remaining: 3m 36s
8:	learn: 0.7606525	test: 0.7571348	best: 0.7571348 (8)	total: 971ms	remaining: 3m 34s
9:	learn: 0.7606727	test: 0.7576199	best: 0.7576199 (9)	total: 1.08s	remaining: 3m 34s
10:	learn: 0.7622089	test: 0.7585092	best: 0.7585092 (10)	total: 1.18s	remaining: 3m 34s
11:	learn: 0.7623706	test: 0.7595602	best:

93:	learn: 0.7887088	test: 0.7838952	best: 0.7838952 (93)	total: 9.99s	remaining: 3m 22s
94:	learn: 0.7888503	test: 0.7838952	best: 0.7838952 (93)	total: 10.1s	remaining: 3m 22s
95:	learn: 0.7892141	test: 0.7843803	best: 0.7843803 (95)	total: 10.2s	remaining: 3m 22s
96:	learn: 0.7898811	test: 0.7848654	best: 0.7848654 (96)	total: 10.3s	remaining: 3m 22s
97:	learn: 0.7899620	test: 0.7849462	best: 0.7849462 (97)	total: 10.4s	remaining: 3m 22s
98:	learn: 0.7902248	test: 0.7852696	best: 0.7852696 (98)	total: 10.5s	remaining: 3m 22s
99:	learn: 0.7901843	test: 0.7860781	best: 0.7860781 (99)	total: 10.6s	remaining: 3m 22s
100:	learn: 0.7902652	test: 0.7858356	best: 0.7860781 (99)	total: 10.8s	remaining: 3m 22s
101:	learn: 0.7902248	test: 0.7857547	best: 0.7860781 (99)	total: 10.9s	remaining: 3m 22s
102:	learn: 0.7905280	test: 0.7855930	best: 0.7860781 (99)	total: 11s	remaining: 3m 21s
103:	learn: 0.7906695	test: 0.7859164	best: 0.7860781 (99)	total: 11.1s	remaining: 3m 21s
104:	learn: 0.79058

184:	learn: 0.8011805	test: 0.7956989	best: 0.7956989 (184)	total: 19.6s	remaining: 3m 12s
185:	learn: 0.8012411	test: 0.7959415	best: 0.7959415 (185)	total: 19.8s	remaining: 3m 12s
186:	learn: 0.8013017	test: 0.7965882	best: 0.7965882 (186)	total: 19.9s	remaining: 3m 12s
187:	learn: 0.8015443	test: 0.7962649	best: 0.7965882 (186)	total: 20s	remaining: 3m 12s
188:	learn: 0.8014230	test: 0.7960223	best: 0.7965882 (186)	total: 20.1s	remaining: 3m 12s
189:	learn: 0.8016049	test: 0.7961032	best: 0.7965882 (186)	total: 20.2s	remaining: 3m 12s
190:	learn: 0.8017667	test: 0.7961032	best: 0.7965882 (186)	total: 20.3s	remaining: 3m 12s
191:	learn: 0.8020294	test: 0.7961032	best: 0.7965882 (186)	total: 20.4s	remaining: 3m 12s
192:	learn: 0.8021103	test: 0.7964266	best: 0.7965882 (186)	total: 20.5s	remaining: 3m 12s
193:	learn: 0.8020901	test: 0.7963457	best: 0.7965882 (186)	total: 20.6s	remaining: 3m 12s
194:	learn: 0.8021911	test: 0.7963457	best: 0.7965882 (186)	total: 20.8s	remaining: 3m 12s
1

275:	learn: 0.8090233	test: 0.8033794	best: 0.8033794 (275)	total: 29.3s	remaining: 3m 2s
276:	learn: 0.8091041	test: 0.8034603	best: 0.8034603 (276)	total: 29.4s	remaining: 3m 2s
277:	learn: 0.8092456	test: 0.8033794	best: 0.8034603 (276)	total: 29.4s	remaining: 3m 2s
278:	learn: 0.8091850	test: 0.8032986	best: 0.8034603 (276)	total: 29.5s	remaining: 3m 2s
279:	learn: 0.8091244	test: 0.8032986	best: 0.8034603 (276)	total: 29.6s	remaining: 3m 2s
280:	learn: 0.8091244	test: 0.8030560	best: 0.8034603 (276)	total: 29.7s	remaining: 3m 1s
281:	learn: 0.8091041	test: 0.8032177	best: 0.8034603 (276)	total: 29.8s	remaining: 3m 1s
282:	learn: 0.8094073	test: 0.8044304	best: 0.8044304 (282)	total: 29.9s	remaining: 3m 1s
283:	learn: 0.8094882	test: 0.8039453	best: 0.8044304 (282)	total: 30s	remaining: 3m 1s
284:	learn: 0.8095286	test: 0.8044304	best: 0.8044304 (282)	total: 30.1s	remaining: 3m 1s
285:	learn: 0.8097510	test: 0.8043496	best: 0.8044304 (282)	total: 30.2s	remaining: 3m 1s
286:	learn: 

366:	learn: 0.8133288	test: 0.8092004	best: 0.8092004 (366)	total: 38.7s	remaining: 2m 52s
367:	learn: 0.8132075	test: 0.8092813	best: 0.8092813 (367)	total: 38.8s	remaining: 2m 52s
368:	learn: 0.8134500	test: 0.8092004	best: 0.8092813 (367)	total: 38.9s	remaining: 2m 51s
369:	learn: 0.8133692	test: 0.8092004	best: 0.8092813 (367)	total: 39s	remaining: 2m 51s
370:	learn: 0.8132479	test: 0.8091196	best: 0.8092813 (367)	total: 39.1s	remaining: 2m 51s
371:	learn: 0.8133085	test: 0.8093621	best: 0.8093621 (371)	total: 39.2s	remaining: 2m 51s
372:	learn: 0.8134096	test: 0.8093621	best: 0.8093621 (371)	total: 39.3s	remaining: 2m 51s
373:	learn: 0.8132681	test: 0.8093621	best: 0.8093621 (371)	total: 39.4s	remaining: 2m 51s
374:	learn: 0.8132883	test: 0.8094430	best: 0.8094430 (374)	total: 39.5s	remaining: 2m 51s
375:	learn: 0.8133490	test: 0.8092004	best: 0.8094430 (374)	total: 39.6s	remaining: 2m 51s
376:	learn: 0.8132075	test: 0.8092004	best: 0.8094430 (374)	total: 39.7s	remaining: 2m 50s
3

...

1631:	learn: 0.8379487	test: 0.8199531	best: 0.8205190 (1614)	total: 2m 46s	remaining: 37.5s
1632:	learn: 0.8379285	test: 0.8199531	best: 0.8205190 (1614)	total: 2m 46s	remaining: 37.3s
1633:	learn: 0.8378477	test: 0.8199531	best: 0.8205190 (1614)	total: 2m 46s	remaining: 37.2s
1634:	learn: 0.8377870	test: 0.8199531	best: 0.8205190 (1614)	total: 2m 46s	remaining: 37.1s
1635:	learn: 0.8377264	test: 0.8200340	best: 0.8205190 (1614)	total: 2m 46s	remaining: 37s
1636:	learn: 0.8376860	test: 0.8200340	best: 0.8205190 (1614)	total: 2m 46s	remaining: 36.9s
1637:	learn: 0.8377668	test: 0.8200340	best: 0.8205190 (1614)	total: 2m 46s	remaining: 36.8s
1638:	learn: 0.8378477	test: 0.8201148	best: 0.8205190 (1614)	total: 2m 46s	remaining: 36.7s
1639:	learn: 0.8378477	test: 0.8200340	best: 0.8205190 (1614)	total: 2m 46s	remaining: 36.6s
1640:	learn: 0.8378477	test: 0.8201148	best: 0.8205190 (1614)	total: 2m 46s	remaining: 36.5s
1641:	learn: 0.8378477	test: 0.8202765	best: 0.8205190 (1614)	total: 2m 

1720:	learn: 0.8387573	test: 0.8205190	best: 0.8206807 (1702)	total: 2m 54s	remaining: 28.3s
1721:	learn: 0.8386966	test: 0.8205190	best: 0.8206807 (1702)	total: 2m 54s	remaining: 28.2s
1722:	learn: 0.8387168	test: 0.8205190	best: 0.8206807 (1702)	total: 2m 54s	remaining: 28.1s
1723:	learn: 0.8386158	test: 0.8206807	best: 0.8206807 (1702)	total: 2m 55s	remaining: 28s
1724:	learn: 0.8385956	test: 0.8206807	best: 0.8206807 (1702)	total: 2m 55s	remaining: 27.9s
1725:	learn: 0.8386562	test: 0.8208424	best: 0.8208424 (1725)	total: 2m 55s	remaining: 27.8s
1726:	learn: 0.8386360	test: 0.8210041	best: 0.8210041 (1726)	total: 2m 55s	remaining: 27.7s
1727:	learn: 0.8386360	test: 0.8210041	best: 0.8210041 (1726)	total: 2m 55s	remaining: 27.6s
1728:	learn: 0.8386158	test: 0.8210850	best: 0.8210850 (1728)	total: 2m 55s	remaining: 27.5s
1729:	learn: 0.8385754	test: 0.8211658	best: 0.8211658 (1729)	total: 2m 55s	remaining: 27.4s
1730:	learn: 0.8385754	test: 0.8211658	best: 0.8211658 (1729)	total: 2m 

1812:	learn: 0.8393435	test: 0.8206807	best: 0.8212467 (1758)	total: 3m 3s	remaining: 19s
1813:	learn: 0.8394041	test: 0.8206807	best: 0.8212467 (1758)	total: 3m 3s	remaining: 18.9s
1814:	learn: 0.8394243	test: 0.8206807	best: 0.8212467 (1758)	total: 3m 3s	remaining: 18.8s
1815:	learn: 0.8395052	test: 0.8207616	best: 0.8212467 (1758)	total: 3m 4s	remaining: 18.7s
1816:	learn: 0.8394647	test: 0.8207616	best: 0.8212467 (1758)	total: 3m 4s	remaining: 18.6s
1817:	learn: 0.8394647	test: 0.8207616	best: 0.8212467 (1758)	total: 3m 4s	remaining: 18.4s
1818:	learn: 0.8394647	test: 0.8206807	best: 0.8212467 (1758)	total: 3m 4s	remaining: 18.4s
1819:	learn: 0.8394850	test: 0.8206807	best: 0.8212467 (1758)	total: 3m 4s	remaining: 18.2s
1820:	learn: 0.8395052	test: 0.8206807	best: 0.8212467 (1758)	total: 3m 4s	remaining: 18.1s
1821:	learn: 0.8394850	test: 0.8206807	best: 0.8212467 (1758)	total: 3m 4s	remaining: 18s
1822:	learn: 0.8394850	test: 0.8206807	best: 0.8212467 (1758)	total: 3m 4s	remaining

1902:	learn: 0.8405158	test: 0.8202765	best: 0.8212467 (1758)	total: 3m 12s	remaining: 9.83s
1903:	learn: 0.8404754	test: 0.8203573	best: 0.8212467 (1758)	total: 3m 12s	remaining: 9.73s
1904:	learn: 0.8405563	test: 0.8201148	best: 0.8212467 (1758)	total: 3m 13s	remaining: 9.63s
1905:	learn: 0.8404956	test: 0.8201148	best: 0.8212467 (1758)	total: 3m 13s	remaining: 9.53s
1906:	learn: 0.8405563	test: 0.8201957	best: 0.8212467 (1758)	total: 3m 13s	remaining: 9.43s
1907:	learn: 0.8405563	test: 0.8202765	best: 0.8212467 (1758)	total: 3m 13s	remaining: 9.32s
1908:	learn: 0.8406371	test: 0.8201148	best: 0.8212467 (1758)	total: 3m 13s	remaining: 9.22s
1909:	learn: 0.8407382	test: 0.8201957	best: 0.8212467 (1758)	total: 3m 13s	remaining: 9.12s
1910:	learn: 0.8407382	test: 0.8201957	best: 0.8212467 (1758)	total: 3m 13s	remaining: 9.02s
1911:	learn: 0.8407584	test: 0.8202765	best: 0.8212467 (1758)	total: 3m 13s	remaining: 8.92s
1912:	learn: 0.8407382	test: 0.8203573	best: 0.8212467 (1758)	total: 3

1991:	learn: 0.8412840	test: 0.8200340	best: 0.8212467 (1758)	total: 3m 21s	remaining: 810ms
1992:	learn: 0.8413446	test: 0.8200340	best: 0.8212467 (1758)	total: 3m 21s	remaining: 709ms
1993:	learn: 0.8412840	test: 0.8201148	best: 0.8212467 (1758)	total: 3m 21s	remaining: 607ms
1994:	learn: 0.8412840	test: 0.8201957	best: 0.8212467 (1758)	total: 3m 21s	remaining: 506ms
1995:	learn: 0.8411829	test: 0.8202765	best: 0.8212467 (1758)	total: 3m 22s	remaining: 405ms
1996:	learn: 0.8411425	test: 0.8202765	best: 0.8212467 (1758)	total: 3m 22s	remaining: 304ms
1997:	learn: 0.8411425	test: 0.8205999	best: 0.8212467 (1758)	total: 3m 22s	remaining: 202ms
1998:	learn: 0.8411020	test: 0.8205999	best: 0.8212467 (1758)	total: 3m 22s	remaining: 101ms
1999:	learn: 0.8412031	test: 0.8203573	best: 0.8212467 (1758)	total: 3m 22s	remaining: 0us

bestTest = 0.821246665
bestIteration = 1758

Shrink model to first 1759 iterations.


In [15]:
metrics.accuracy_score(y_val, model.predict(X_val))

0.8212466650497211

In [16]:
# vl = pd.DataFrame([X_train.columns, model.get_feature_importance()]).T
# vl.columns = ['Feature', 'Importance']
# vl = vl.sort_values('Importance', ascending=False)
# vl

In [17]:
import pickle

with open(r'C:\Users\minht\Desktop\MLOps\src\phase2\models\problem2.pkl', 'wb') as f:
    pickle.dump(model, f)